## Steps To be Followed
1. Import Necessary Libraries
2. Create S3 Bucket
3. Mapping The path of the models in S3
4. Mapping train And Test Data in S3

### 1.Import Libraries

In [2]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input,Session

### 2. Create S3 Bucket

In [22]:
bucket_name = 'bankapplication-praveen'

#Checking the region name
my_region = boto3.session.Session().region_name
print(my_region)

ca-central-1


In [23]:
### Get access of S3
s3 = boto3.resource('s3')

try:
    if my_region == 'ca-central-1':
#         s3.create_bucket(bucket_name=bucket_name)
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={
    'LocationConstraint': 'ca-central-1'})
    print('S3 bucket created successfully')

except Exception as e:
    print('S3 error:',e)

S3 bucket created successfully


In [24]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  bankapplication-praveen


### 3. Map the Model output Path

In [26]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path) # Every model trained will be stored in the output folder

s3://bankapplication-praveen/xgboost-as-a-built-in-algo/output


### 4.Downloading The Dataset and Storing in S3 Bucket

In [30]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: Donwloaded bank_clean.csv.')
     
except Exception as e:
    print('Data load error: ',e)
        
try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Sucess: Data loaded into dataframe.')
    
except Exception as e:
    print('Data load error: ',e)

Success: Donwloaded bank_clean.csv.
Sucess: Data loaded into dataframe.


In [31]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


### 5. Split Train and Test Sets

In [34]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape,test_data.shape)

(28831, 61) (12357, 61)


### 6.Saving to Bucket

In [36]:
import os
 
# Training Data    
    
# AWS needs to have the dependent feature as the first column
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
# Upload into the Bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
# Setting the training path
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


In [39]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### 7.Building Models Xgboot- Inbuilt Algorithm

Note: These models are stored as an Image or Container

In [40]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [41]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [49]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

### 8. Fit the model

In [50]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-03-21-04-36-51-131


2023-03-21 04:36:51 Starting - Starting the training job...
2023-03-21 04:37:13 Starting - Preparing the instances for training.........
2023-03-21 04:38:27 Downloading - Downloading input data...
2023-03-21 04:39:08 Training - Training image download completed. Training in progress...[2023-03-21 04:39:31.671 ip-10-0-78-195.ca-central-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single n

### 9. Deploy

In [51]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-03-21-04-51-53-342
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-03-21-04-51-53-342
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-03-21-04-51-53-342


------!

### 10. Prediction of the Test Data

In [60]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
# xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


### 11. Evaluation

In [66]:
# Confusion Matrix
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))
     


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



### 12. Deleting the Endpoints

In [68]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-03-21-04-51-53-342


[{'ResponseMetadata': {'RequestId': 'GZHE7EJZ6PP7HXDM',
   'HostId': 'im0U5qIKii8lXdrrG1/vREbUshXOLxe4Gu3MD7b3fhshdwYo9g0ee5x+nLWaL7oFnVG5RnsY8ew=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'im0U5qIKii8lXdrrG1/vREbUshXOLxe4Gu3MD7b3fhshdwYo9g0ee5x+nLWaL7oFnVG5RnsY8ew=',
    'x-amz-request-id': 'GZHE7EJZ6PP7HXDM',
    'date': 'Tue, 21 Mar 2023 05:13:36 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-03-21-04-36-51-131/profiler-output/system/incremental/2023032104/1679373480.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-03-21-04-36-51-131/debug-output/collections/000000000/worker_0_collections.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-03-21-04-36-51-131/debug-output/index/000000000/000000000020_worker_0.json'},
